In [114]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.cbook
import pandas as pd
from scipy.ndimage import gaussian_filter1d
import pickle
import sys
sys.path.append("../helpers/")

from unit_selectivity import *

import json_tricks as jt

import scipy.stats as sts

# warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
# warnings.filterwarnings("ignore",category=UserWarning)
# np.set_printoptions(suppress=True)



# load data

In [115]:

#a1 cells
with open('../Data/all_cells_25ms.json', 'r') as pf:
    a1 = jt.load(pf)

with open('../Data/a1_errors_25ms.json', 'r') as pf:
    a1_err = jt.load(pf)

# pfc cells
with open('../Data/pfc_25ms.json', 'r') as pf:
    pfc = jt.load(pf)

with open('../Data/pfc_errors_25ms.json', 'r') as pf:
    pfc_err = jt.load(pf)



In [116]:
pfc_err[0].keys()

odict_keys(['Summed', 'stim', 'binsize', 'time', 'region', 'is_error', 'is_wrong', 'is_false', 'choice_made', 'cpoke_start', 'cpoke_stop', 'rat', 'unit', 'ulabel', 'correct', 'hi_pc_no', 'le_hi_lc', 'le_hi_pc', 'le_lc_go', 'le_lo_lc', 'le_lo_pc', 'lo_pc_go', 'ri_hi_lc', 'ri_hi_pc', 'ri_lc_no', 'ri_lo_lc', 'ri_lo_pc', 'le_lc', 'le_pc', 'ri_lc', 'ri_pc', 'lo_lc', 'lo_pc', 'hi_lc', 'hi_pc'])

# calculate bining - nasty, nasty function
### needs refactoring 

In [117]:
def calculate_bins(ac):
    selectivity, pop_ids, _ = get_select_units(ac)

    pop_ids = np.array(pop_ids)

    pop_ids[pop_ids == 'none'] = 0
    pop_ids[pop_ids == 'lc'] = 1
    pop_ids[pop_ids == 'pc'] = 2

    pop_ids = np.array(pop_ids,dtype="int")

    time= ac[0]["time"]
    stims = np.unique(ac[0]["stim"])

    # remove transition trials
    stims_idx = np.array([s[-1] == "o" for s in stims])
    stims = stims[~stims_idx]

    sessions_ids = ["_".join(a["ulabel"].split("_")[:3]) for a in ac]


    all_spike_counts = []
    all_neuron_ids = []
    all_neuron_labels = []
    all_population_ids = []
    all_stims = []
    all_sessions_ids = []
    all_contexts = []
    all_IA = []
    all_IB = []
    all_decisions = []

    for neuron_id in range(len(ac)):
        neuron_data = ac[neuron_id]
        population_id = pop_ids[neuron_id]
        sessions_id = sessions_ids[neuron_id]

        all_spike_counts_neuron = []
        for stim in stims:
            decision = 0
            if (stim == "le_lo_lc") or (stim == "le_hi_lc"):
                decision = -1
            if (stim == "ri_lo_pc") or (stim == "le_lo_pc"):
                decision = 1
            
            spike_counts = neuron_data[stim]
            neuron_ids = [neuron_id]*len(spike_counts)
            neuron_label = [neuron_data["ulabel"]]*len(spike_counts)
            population_ids = [population_id]*len(spike_counts)
            stims_neuron = [stim]*len(spike_counts)
            sessions_neuron = [sessions_id]*len(spike_counts)
            contexts = np.array([stim.split("_")[-1]]*len(spike_counts))
            decisions = np.array([decision]*len(spike_counts))
            contexts[contexts == 'none'] = 0
            contexts[contexts == 'lc'] = 1
            contexts[contexts == 'pc'] = 2
            IA,IB = np.array([[s.split("_")[0]=="ri",s.split("_")[1]=="lo"] for s in stims_neuron],dtype="int").T

            all_decisions.append(decisions)
            all_spike_counts_neuron.append(spike_counts)
            all_neuron_ids.append(neuron_ids)
            all_neuron_labels.append(neuron_label)
            all_population_ids.append(population_ids)
            all_stims.append(stims_neuron)
            all_sessions_ids.append(sessions_neuron)
            all_contexts.append(contexts)
            all_IA.append(IA)
            all_IB.append(IB)

        counts = np.concatenate(all_spike_counts_neuron)
        #zscores all neuron's firing rate. Not how Remi does.
        #counts = (counts - np.mean(counts)) / np.std(counts)
        all_spike_counts.append(counts)

    # convert to right dtype
    all_decisions = np.concatenate(all_decisions).astype("int")
    all_neuron_ids = np.concatenate(all_neuron_ids).astype("int")
    all_neuron_labels = np.concatenate(all_neuron_labels).astype("str")
    all_sessions_ids = np.concatenate(all_sessions_ids).astype("str")
    all_population_ids = np.concatenate(all_population_ids).astype("int")
    all_stims = np.concatenate(all_stims).astype("str")
    all_contexts = np.concatenate(all_contexts).astype("int")
    all_spike_counts = np.concatenate(all_spike_counts).astype("int")
    all_IA = np.concatenate(all_IA).astype("int")
    all_IB = np.concatenate(all_IB).astype("int")


    bins = ["bin_"+str(i) for i in range(len(time))]

    df = pd.DataFrame()
    df["neuron_id"] = all_neuron_ids
    df["neuron_label"] = all_neuron_labels
    df["sessions_id"] = all_sessions_ids
    df["population_id"] = all_population_ids
    df["stims"] = all_stims
    df["context"] = all_contexts
    df["pre_stim"] = np.mean(all_spike_counts[:,time<0],1)
    df["IA"] = all_IA
    df["IB"] = all_IB
    df["decision"] = all_decisions

    for bi,b in enumerate(bins):
        df[b] = all_spike_counts[:,bi]

    return df





In [123]:

df_a1 = calculate_bins(a1)
df_a1_err = calculate_bins(a1_err)

# flip answers for error trials - weirdly all errors were like this
df_a1_err.loc[(df_a1_err.context == 1),"decision"]  = (df_a1_err[(df_a1_err.context == 1)].decision + 1)*-1
df_a1_err.loc[(df_a1_err.context == 2),"decision"] = (df_a1_err[(df_a1_err.context == 2)].decision - 1)*-1

# add error labels and concat
df_a1.insert(10, 'correct', 1)
df_a1_err.insert(10, 'correct', 0)
df_a1 = pd.concat([df_a1,df_a1_err])

labels = df_a1["neuron_label"]
for li,label in enumerate(labels.unique()):
    df_a1.loc[df_a1["neuron_label"]==label,"neuron_id"] = li


df_pfc = calculate_bins(pfc)
df_pfc_err = calculate_bins(pfc_err)

# flip answers for error trials - weirdly all errors were like this
df_pfc_err.loc[(df_pfc_err.context == 1),"decision"]  = (df_pfc_err[(df_pfc_err.context == 1)].decision + 1)*-1
df_pfc_err.loc[(df_pfc_err.context == 2),"decision"] = (df_pfc_err[(df_pfc_err.context == 2)].decision - 1)*-1

# add error labels and concat
df_pfc.insert(10, 'correct', 1)
df_pfc_err.insert(10, 'correct', 0)
df_pfc = pd.concat([df_pfc,df_pfc_err])

labels = df_pfc["neuron_label"]
for li,label in enumerate(labels.unique()):
    df_pfc.loc[df_pfc["neuron_label"]==label,"neuron_id"] = li

In [124]:
df_a1.to_csv("../Data/RW_dataframe_A1_25ms_w_errors.csv",index=False)
df_pfc.to_csv("../Data/RW_dataframe_PFC_25ms_w_errors.csv",index=False)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=af3700bb-635f-4f56-95cc-6d24931ab29c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>